# Backtesting Using BackTrader

In [1]:
import backtrader as bt
import datetime
import pandas as pd
import numpy as np

In [2]:
class PandasData(bt.feeds.PandasData):
    lines = ('open', 'close',)
    params = (
        ('datetime', None),  # use index as datetime
        ('open', 0),         # the [0] column is open price
        ('close', 1),        
        ('high', -1),
        ('low', -1),
        ('volume', -1),
        ('openinterest', -1),
    )

In [14]:
class BLStrategy(bt.Strategy):
    # list for tickers
    params = (
        ('stocks', []),
    )


    def __init__(self, weights):
        # init data feeds and orders
        self.datafeeds = {}
        self.orders = {}

        # price data for all stocks
        for i, ticker in enumerate(self.params.stocks):
            self.datafeeds[ticker] = self.datas[i]

        # weights for all stocks
        self.weights = weights


    # log the orders
    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                print(f"Bought {order.executed.size} shares of {order.data._name} at {order.executed.price}")
            elif order.issell():
                print(f"Sold {order.executed.size} shares of {order.data._name} at {order.executed.price}")
                
    
    # for each date, place orders according to the weights, this part has some bugs
    def next(self):
        date = self.data.datetime.date(0)
        weights = self.weights.loc[date.strftime('%Y-%m-%d')]

        total_cash = self.broker.get_cash()
        print(f"Processing Date: {date} with total cash: {total_cash}")

        total_value = self.broker.getvalue()  # includes cash and current market value of stocks

        for ticker in self.params.stocks:
            data = self.datafeeds[ticker]
            target_percent = weights[ticker]
            # this handles both long and short positions
            self.order_target_percent(data, target=target_percent)

        print(f"End of Day Portfolio Value: {self.broker.getvalue()}")
        print("===============================================")


if __name__ == '__main__':
    # load price and weights data
    close_prices_df = pd.read_csv('../data/synthetic_close_prices.csv', index_col='Date', parse_dates=True)
    open_prices_df = pd.read_csv('../data/synthetic_open_prices.csv', index_col='Date', parse_dates=True)
    weights_df = pd.read_csv('../data/synthetic_weights.csv', index_col='Date', parse_dates=True)
    
    # Combine open and close prices into one DataFrame
    combined_df = open_prices_df.join(close_prices_df, lsuffix='_open', rsuffix='_close')
    combined_df = combined_df.dropna()
    combined_df = combined_df.head()
    display(combined_df.head())
    
    # align the date of price and weights
    weights_df = weights_df.loc[combined_df.index]

    # initialize cerebro engine
    cerebro = bt.Cerebro()

    # read data feeds
    for col in close_prices_df.columns:
        data = PandasData(dataname=combined_df[[col + '_open', col + '_close']])
        cerebro.adddata(data, name=col)

    # strategy setting
    cerebro.broker.set_cash(100000)
    cerebro.broker.setcommission(commission=0.001)
    cerebro.broker.set_shortcash(True)
    cerebro.addstrategy(BLStrategy, weights=weights_df, stocks=close_prices_df.columns)

    cerebro.run()

    print('Final Portfolio Value:', cerebro.broker.getvalue())


,Stock1_open,Stock2_open,Stock3_open,Stock4_open,Stock5_open,Stock1_close,Stock2_close,Stock3_close,Stock4_close,Stock5_close
Date,,,,,,,,,,
2024-01-01,107.725796,105.955675,96.894568,105.037252,101.133499,102.105190,101.524408,88.194448,102.723578,84.726993
2024-01-02,108.124750,108.325321,88.165682,98.113114,115.012790,95.098102,129.469338,86.574331,102.046604,86.722759
2024-01-03,97.561152,112.002587,100.638903,100.729488,104.055575,101.660345,84.821986,100.573143,110.831303,89.813587
2024-01-04,90.235622,106.391998,108.718087,89.276592,101.530093,84.818717,102.548228,93.151530,89.970786,92.284696
2024-01-05,105.030983,101.710949,118.171227,101.476200,83.623055,88.758674,96.018422,91.592941,85.661333,116.130754


Processing Date: 2024-01-01 with total cash: 100000.0
End of Day Portfolio Value: 100000.0
Sold -7154 shares of Stock1 at 108.12475044232652
Sold -7737 shares of Stock2 at 108.32532054559624
Bought 9268 shares of Stock3 at 88.16568220987148
Bought 2893 shares of Stock4 at 98.11311412031058
Processing Date: 2024-01-02 with total cash: 607964.089602938
End of Day Portfolio Value: 23519.72940651735
Bought 6303 shares of Stock1 at 97.56115157339828
Processing Date: 2024-01-03 with total cash: -7578.776702558394
End of Day Portfolio Value: 502387.4159145169
Sold -12134 shares of Stock3 at 108.71808730434856
Sold -54 shares of Stock4 at 89.27659183539471
Processing Date: 2024-01-04 with total cash: 1315103.4244002083
End of Day Portfolio Value: 437961.83454641385
Bought 3042 shares of Stock1 at 105.0309826683512
Sold -5490 shares of Stock4 at 101.47619974632056
Processing Date: 2024-01-05 with total cash: 764102.3512461616
End of Day Portfolio Value: 468979.0413812473
Final Portfolio Value: 

In [12]:
# date range
start_date = datetime.date(2024, 1, 1)
end_date = start_date + datetime.timedelta(days=99)
date_range = pd.bdate_range(start=start_date, end=end_date)

# fake price for 5 stocks
num_days = len(date_range)
num_stocks = 5
prices = pd.DataFrame(np.random.normal(loc=100, scale=10, size=(num_days, num_stocks)), index=date_range, columns=[f'Stock{i}' for i in range(1, num_stocks+1)])
prices = prices.reset_index().rename(columns={"index": "Date"})

# fake weights for 5 stocks
weights = np.random.uniform(-1, 1, (num_days, num_stocks))
weights = weights / weights.sum(axis=1, keepdims=True)
weights = pd.DataFrame(weights, index=date_range, columns=[f'Stock{i}' for i in range(1, num_stocks+1)])
weights = weights.reset_index().rename(columns={"index": "Date"})


# save price and weights
# prices.to_csv('../data/synthetic_open_prices.csv')
# weights.to_csv('../data/synthetic_weights.csv')

print("Data generated and saved successfully.")

print(prices.head())
print(weights.head())

Data generated and saved successfully.
        Date      Stock1      Stock2      Stock3      Stock4      Stock5
0 2024-01-01   89.278044   72.770513   92.084425  107.102840   87.580582
1 2024-01-02  107.708380  110.863840  119.725076   94.094580   88.800516
2 2024-01-03   84.115801   99.211717   99.890973   88.683592   96.493743
3 2024-01-04  104.471406   93.874793   91.871149  104.149267  101.779381
4 2024-01-05   83.653062   91.563991  111.332034  105.648394  103.329560
        Date    Stock1    Stock2    Stock3    Stock4    Stock5
0 2024-01-01  0.391449 -0.097651  0.388190  0.019484  0.298529
1 2024-01-02  0.064482  1.565104 -0.925339  2.035023 -1.739270
2 2024-01-03 -1.439431  1.297013  0.181790  1.186281 -0.225653
3 2024-01-04 -0.547698  0.523858  0.702089  0.602619 -0.280867
4 2024-01-05  0.337418  0.183443  0.427368 -0.328869  0.380640
